In [1]:
import tensorflow as tf
import pickle
import pandas as pd
import matplotlib.pyplot as plt
from util import *
from load_data import load_input_file, build_combined_input
from NN_train import *
path_tosave = 'Models/1stDiscoModel'
features_to_ignore = []
#features_to_ignore = ["DPhill_SS"]
features_to_ignore = ["DRll01","HT_jets","sumPsbtag","met_met","Mll01","DEtall_SS", "jet_pseudoscore_DL1r2","nJets_OR_DL1r_77","sumPsbtag77"]

In [2]:
samples, feature_names, class_names, class_no = load_input_file('data/sstt_multiclass_background_V03.h5', features_to_ignore)
input_features, targets, class_labels, weights = build_combined_input(samples)

Ignoring feature:  DRll01
Ignoring feature:  HT_jets
Ignoring feature:  sumPsbtag
Ignoring feature:  met_met
Ignoring feature:  Mll01
Ignoring feature:  DEtall_SS
Ignoring feature:  jet_pseudoscore_DL1r2
Ignoring feature:  nJets_OR_DL1r_77
Ignoring feature:  sumPsbtag77
Found 17 features on the dataset 8 features were loaded: 

DPhill_SS
HT_lep
MtLepMet
jet_pseudoscore_DL1r0
jet_pseudoscore_DL1r1
jet_pt0_nofwd
nJets_OR
randomRunNumber
 
Dataset contains extra labels for different backgrounds!
Class 0  :  679239
Class 1  :  231587
Class 2  :  3249
Class 3  :  310859
Class 4  :  190589
Class 5  :  223033
Class 6  :  355647
Class 7  :  109991
Class 8  :  1039
Dataset contains 679239 Signal events and 1425994 Background events.


In [3]:
DPhi = np.squeeze(input_features[:,0:1])
Number = np.squeeze((input_features[:,7:8]).astype(int))
#Number = np.arange(0,1679947)
feature_names = feature_names[1:7]
input_features = input_features[:,1:7]

In [4]:
index_sort = np.argsort(class_no)
class_names = (np.array(class_names)[index_sort])
class_no = np.sort(class_no)
print(class_names)
print(class_no)
No_Classes = class_no.max()+1

['signal' 'ttZ' 'tty' 'ttW' 'tttt' 'ttH' 'Other' 'Diboson' 'Fakes']
[0 1 2 3 4 5 6 7 8]


In [5]:
ScaleWeights(targets,weights)
#scale the input between 0-1
scaler = StandardScaler().fit(input_features)
input_scaled = scaler.transform(input_features)


======Weight Statistic========================================
Weights::        W(1)=1200.48, W(0)=2983.69
Scaled weights:: W(1)=1.05262e+06, W(0)=1.05262e+06


In [6]:
def Fold_Odd_Even(input, targets, weights, class_labels, Number, Dphi):
    input_odd = input[Number % 2 == 1]
    targets_odd = targets[Number % 2 == 1]
    weights_odd = weights[Number % 2 == 1]
    class_labels_odd = class_labels[Number % 2 == 1]
    input_even = input[Number % 2 == 0]
    targets_even = targets[Number % 2 == 0]
    weights_even = weights[Number % 2 == 0]
    class_labels_even = class_labels[Number % 2 == 0]
    Dphi_even = Dphi[Number % 2 == 0]
    Dphi_odd = Dphi[Number % 2 == 1]
    return (input_odd, targets_odd, weights_odd, class_labels_odd, Dphi_odd), (input_even, targets_even, weights_even, class_labels_even, Dphi_even)

In [7]:
odd, even = Fold_Odd_Even(input_scaled, targets, weights, class_labels, Number, DPhi)

In [20]:
#train, val, test = Train_Val_Test_Split(input_scaled, targets, weights, class_labels)
input_shape = odd[0].shape[1]
n_epochs = 30
batch_size = 20000

In [40]:
def Create_Model_with_Disco(input_shape):
    layer_opts = dict( activation = 'sigmoid', kernel_initializer = initializers.glorot_normal(seed=seed))
    Dphi_and_weight = Kl.Input(shape = (2,))
    input_layer = Kl.Input(shape = input_shape )
    x = Kl.Dense( 36, **layer_opts) (input_layer)
    x = Kl.Dense( 48, **layer_opts) (x)
    y_pred = Kl.Dense( 1., activation = 'sigmoid', name = "OutputLayer" )(x)
    model = Km.Model(inputs= [input_layer, Dphi_and_weight], outputs=y_pred)
    model_optimizer = Adam(lr=0.0001)
    model_disco = Loss_Disco(Dphi_and_weight, 0.1)
    model.compile(optimizer=tf.keras.optimizers.Adam(),loss=model_disco, metrics = ['accuracy'])
    model.summary()
    return model

In [36]:
def Train_Odd_Even(odd, even, n_epochs = 400, batch_size = 2000):
    fit_history_list = []
    model_list = []
    input_shape = odd[0].shape[1]

    lr_schedule = tf.keras.callbacks.LearningRateScheduler(lr_step_decay)

    X_odd, X_val_odd, weights_odd, weights_val_odd, y_odd, y_val_odd, Dphi_odd, Dphi_val_odd = train_test_split(odd[0], odd[2], odd[1], odd[4], test_size=0.2)
    X_even, X_val_even, weights_even, weights_val_even, y_even, y_val_even, Dphi_even, Dphi_val_even = train_test_split(even[0], even[2], even[1], even[4], test_size=0.2)
    
    Dphi_and_weights_odd = np.vstack([Dphi_odd, weights_odd]).T
    Dphi_and_weights_even = np.vstack([Dphi_even, weights_even]).T
    
    Dphi_and_weights_odd_val = np.vstack([Dphi_val_odd, weights_val_odd]).T
    Dphi_and_weights_even_val = np.vstack([Dphi_val_even, weights_val_even]).T
    

    test = [X_odd, Dphi_and_weights_odd]
    print(test[0].shape)
    print(test[1].shape)
    
    model_odd = Create_Model_with_Disco(input_shape)
    model_even = Create_Model_with_Disco(input_shape)
    fit_history_odd = model_odd.fit([X_odd, Dphi_and_weights_odd], y_odd, epochs = n_epochs, shuffle = True, batch_size = batch_size, validation_data=([X_val_odd,Dphi_and_weights_odd_val], y_val_odd, weights_val_odd), sample_weight=weights_odd, verbose=0 ,callbacks=[tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 100, verbose = True, min_delta = 0.001),lr_schedule])
    fit_history_even = model_even.fit([X_even, Dphi_and_weights_even], y_even, epochs = n_epochs, shuffle = True, batch_size = batch_size, validation_data=([X_val_even,Dphi_and_weights_even_val], y_val_even, weights_val_even), sample_weight=weights_even, verbose=0 ,callbacks=[tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 100, verbose = True, min_delta = 0.001),lr_schedule])

    return fit_history_odd, model_odd, fit_history_even, model_even

In [39]:
hist_odd, model_odd, hist_even, model_even = Train_Odd_Even(odd, even, n_epochs = n_epochs, batch_size = batch_size)

(857245, 6)
(857245, 2)
Model: "model_15"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_32 (InputLayer)           [(None, 6)]          0                                            
__________________________________________________________________________________________________
dense_30 (Dense)                (None, 36)           252         input_32[0][0]                   
__________________________________________________________________________________________________
dense_31 (Dense)                (None, 48)           1776        dense_30[0][0]                   
__________________________________________________________________________________________________
input_31 (InputLayer)           [(None, 2)]          0                                            
___________________________________________________________________

InvalidArgumentError: Can not squeeze dim[0], expected a dimension of 1, got 20000
	 [[{{node loss_22/OutputLayer_loss/weighted_loss/Squeeze}}]]

In [ ]:
history_list = [hist_odd, hist_even]
Plot_Metrics_KFold(history_list, path_tosave)

In [ ]:
Save_Model(model_odd, path_tosave, "_odd")
Save_Model(model_even, path_tosave, "_even")

pickle.dump(scaler, open(path_tosave + '/scaler.pkl','wb'))

# Evaluation

model_odd = Load_Model(path_tosave, name="_odd")
model_even = Load_Model(path_tosave, name="_even")

In [ ]:
Plot_NN_Output(model_odd, odd, even, path_tosave, True)

In [ ]:
Plot_NN_Output(model_even, even, odd, path_tosave, True)

In [ ]:
plot_roc_curve(model_even, odd, path_tosave)
plot_roc_curve(model_odd, even, path_tosave)

In [ ]:
nn_scores_odd = model_even.predict(odd)
nn_scores_even = model_odd.predict(even)

fig = plt.figure(figsize=(16,8))
plt.grid(color='k', which='both', linestyle='--', lw=0.5, alpha=0.1, zorder = 0)
plt.xlabel("NN output", horizontalalignment='right', x=1)
plt.xlim([0,1])
plt.ylabel("Density")
plt.yscale('log')
histargs = {"bins":40, "range":(0,1.), "density":True, "histtype":'step'}
for i in range(No_Classes):
    plt.hist(nn_scores_test[test[3]==i],label = class_names[i], **histargs)
plt.legend(loc='upper center')
    #plt.hist(nn_scores_test[test[1]==0][test[3]==i],label = "Test_Background", **histargs)
plt.savefig(path_tosave+"/NN_Output_Classes.png")

# Input Correlation

In [ ]:
corr_mat = np.corrcoef(input_features, None, False)
plt.rcParams["figure.figsize"] = (12,12)
sns.heatmap(corr_mat, annot=True, cmap="viridis", xticklabels=feature_names, yticklabels=feature_names, fmt=".2f")
plt.savefig(path_tosave+"/Corr_Mat.png")

# Calc Signal cut for desired Singal Efficiency

In [ ]:
Sig = nn_scores_odd[odd[1]==1]
Bkg = nn_scores_odd[odd[1]==0]
Sig_Eff, Bkg_Eff, Signal_Cut = Find_Eff_Cut(Sig=Sig, Bkg=Bkg, Signal_Eff=0.8)

# Feature Importance

In [ ]:
f, g = get_feature_importance(even, model_odd, Signal_Cut, 1)
idx = np.argsort(f)
fig, ax = plt.subplots(figsize=(20, 10))
ax.barh(range(even[0].shape[1]), np.sort(f), xerr=np.array(g)[idx], color="r", alpha=0.7, ecolor='black', capsize=10)
ax.set_yticks(range(even[0].shape[1]), np.array(feature_names)[idx])
ax.set_xlabel('Feature Importance')
plt.savefig(path_tosave + "/Feature_Importance_even.png")

In [ ]:
f, g = get_feature_importance(odd, model_even, Signal_Cut, 1)
idx = np.argsort(f)
fig, ax = plt.subplots(figsize=(20, 10))
ax.barh(range(even[0].shape[1]), np.sort(f), xerr=np.array(g)[idx], color="r", alpha=0.7, ecolor='black', capsize=10)
ax.set_yticks(range(even[0].shape[1]), np.array(feature_names)[idx])
ax.set_xlabel('Feature Importance')
plt.savefig(path_tosave + "/Feature_Importance_odd.png")

# Confusion Matrix

In [ ]:
Sig = nn_scores_even[even[1]==1]
Bkg = nn_scores_even[even[1]==0]
Sig_Eff, Bkg_Eff, Signal_Cut_even = Find_Eff_Cut(Sig=Sig, Bkg=Bkg, Signal_Eff=0.8)

In [ ]:
Make_Confusion_Matrix(even, nn_scores_even, Signal_Cut_even, class_names, path_tosave, False)

In [ ]:
Make_Confusion_Matrix(even, nn_scores_even, Signal_Cut_even, class_names, path_tosave, True)

In [ ]:
Make_Confusion_Matrix(odd, nn_scores_odd, Signal_Cut, class_names, path_tosave, True)

In [ ]:
Make_Confusion_Matrix(odd, nn_scores_odd, Signal_Cut, class_names, path_tosave, False)